In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from dotenv import load_dotenv
load_dotenv()
PWD = os.getenv('PWD')
db_name = PWD+'\\database'+'\\RVNUSDT.db'
import sys
sys.path.insert(1, PWD+'\\modules')

In [2]:
df = pd.read_sql(
    '''
        SELECT * FROM m5 
        WHERE date_created > "2021-12-00 00:00:00.000000"
    ''',
    'sqlite:///' + db_name, 
    index_col='id',
)
len(df)

8800

In [6]:
# %%timeit -n 10
for i, row in df.iterrows():
    df.loc[i, 'amplitude'] = (float(row['high_']) - float(row['low_']))/float(row['low_'])*100

In [18]:
%%timeit -n 10
for i, row in df.iterrows():
    df.loc[i, 'amplitude'] = (np.float64(row['high_']) - np.float64(row['low_']))/np.float64(row['low_'])*100

3.96 s ± 82.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%%timeit 
for i, row in df.iterrows():
    df.loc[i, 'amplitude'] = (np.float64(row['high_']) - np.float64(row['low_']))/np.float64(row['low_'])*np.uint8(100)

3.82 s ± 99.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
from numba import njit

# @njit
# # def njit_():
# for i, row in df.iterrows():
#     df.loc[i, 'amplitude'] = (np.float64(row['high_']) - np.float64(row['low_']))/np.float64(row['low_'])*np.uint8(100)

In [3]:

import numba


@numba.njit
def amplitude_f_numba(a, b, ):
    return (a - b) / b * 100
    # return (np.float64(a) - np.float64(b))/np.float64(b)*np.uint8(100)


@numba.njit
def apply_amplitude_f_numba(col_high_, col_low_, ):
    n = len(col_high_)
    result = np.empty(n, dtype="float64")
    assert len(col_high_) == len(col_low_) == n
    for i in range(n):
        result[i] = amplitude_f_numba(col_high_[i], col_low_[i], )
    return result


def compute_numba(df):
    result = apply_amplitude_f_numba(
        df["high_"].to_numpy(dtype='float64'), 
        df["low_"].to_numpy(dtype='float64'), 
    )
    df['amplitude'] = pd.Series(result, index=df.index, )
    return df


In [ ]:
import cProfile

In [9]:
# %%timeit -n 100
%prun compute_numba(df)

         468 function calls (460 primitive calls) in 0.007 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.005    0.001    0.005    0.001 {built-in method numpy.array}
        1    0.000    0.000    0.001    0.001 blocks.py:2711(make_block)
        1    0.000    0.000    0.000    0.000 managers.py:1057(iset)
        1    0.000    0.000    0.000    0.000 3413057701.py:10(apply_amplitude_f_numba)
        1    0.000    0.000    0.007    0.007 3413057701.py:20(compute_numba)
        2    0.000    0.000    0.000    0.000 managers.py:998(iget)
        2    0.000    0.000    0.001    0.000 frame.py:2987(__getitem__)
        3    0.000    0.000    0.001    0.000 series.py:238(__init__)
        2    0.000    0.000    0.000    0.000 generic.py:3783(_get_item_cache)
       76    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.007    0.007 {built-in method builtins.

In [5]:
df = compute_numba(df)
df

,date_created,open_time,open_,high_,low_,close_,volume_,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore_,amplitude
id,,,,,,,,,,,,,,
230292,2021-12-01 00:00:00.000000,1638306000000,0.10843000,0.10899000,0.10841000,0.10882000,383820.00000000,1638306299999,41754.49734100,91,263645.70000000,28680.24724400,0,0.535006
230293,2021-12-01 00:05:00.000000,1638306300000,0.10882000,0.10884000,0.10836000,0.10849000,226955.20000000,1638306599999,24651.80424200,89,82917.70000000,9009.24099600,0,0.442968
230294,2021-12-01 00:10:00.000000,1638306600000,0.10850000,0.10866000,0.10849000,0.10863000,242321.70000000,1638306899999,26306.41257500,99,151675.20000000,16465.62420300,0,0.156696
230295,2021-12-01 00:15:00.000000,1638306900000,0.10863000,0.10888000,0.10850000,0.10856000,221322.50000000,1638307199999,24050.57299600,93,140344.10000000,15249.74851800,0,0.350230
230296,2021-12-01 00:20:00.000000,1638307200000,0.10856000,0.10862000,0.10840000,0.10843000,906280.40000000,1638307499999,98337.47837400,150,295507.90000000,32068.47470500,0,0.202952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239087,2021-12-31 09:20:00.000000,1640931600000,0.09453000,0.09492000,0.09446000,0.09487000,317678.80000000,1640931899999,30068.68891200,160,234649.00000000,22216.13319500,0,0.486979
239088,2021-12-31 09:25:00.000000,1640931900000,0.09490000,0.09501000,0.09474000,0.09480000,166964.50000000,1640932199999,15836.15077100,94,45538.50000000,4321.94835700,0,0.284991
239089,2021-12-31 09:30:00.000000,1640932200000,0.09477000,0.09502000,0.09476000,0.09501000,133066.90000000,1640932499999,12633.91783700,90,61023.20000000,5794.23222500,0,0.274377
